### Meeting Prep Agent
In this lesson, we will be exploring the creation of an AI agent for meeting preparation. We would be using this example to study key concepts within crew AI


#### Tool definition
The first step when building an agent from a development standpoint is to define the tools that would be used by the agent


In [2]:
import os
from exa_py import Exa
from langchain.agents.tools import tool
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

from langchain.tools import Tool

wikitool = Tool(
    name="search",
    description="Find relevant persons and information from the web",
    func=wikipedia.run
)

In [4]:
class ExaSearchTool:
    @tool
    def search(query: str):
        """
        Search for relevant webpages given a query
        """
        return ExaSearchTool._exa().search(f"{query}", use_autoprompt=True, num_results=3)
    
    @tool
    def find_similar(url: str):
        """ 
        Search for similar webpages given a url
        """
        return ExaSearchTool._exa().find_similar(url, num_results=3)
    @tool
    def get_contents(ids: str):
        """Get the contents of a webpage.
		The ids must be passed in as a list, a list of ids returned from `search`.
		"""
        ids = eval(ids)
        contents: str =  (ExaSearchTool._exa().get_contents(ids))
        print(contents)
        contents = contents.split("URL:")
        contents = [content[:1000] for content in contents]
        return "\n\n".join(contents)

    @staticmethod
    def tools():
        return [ExaSearchTool.search, ExaSearchTool.find_similar, ExaSearchTool.get_contents]
    @staticmethod
    def _exa():
        return Exa(api_key=os.environ.get("EXA_API_KEY"))

##### Defining an Agent
Once the tools have been defined, you can then define the agent

In [4]:
from textwrap import dedent
from crewai import Agent

In [6]:
class MeetingPrepAgents():
    def research_agent(self):
        return Agent(
            role="Research Specialist",
            goal="Conduct research on people and companies involved in the meeting",
            tools=[wikitool],
            backstory=dedent("""
                             As a Research Specialist, your mission is to uncover detailed information
                             about the individuals and entities participating in the meeting. Your insights
                             will lay the groundwork for strategic meeting preparation.
                             """),
            verbose=True
        )
    
    def industry_analysis_agent(self):
        return Agent(
            role="Industry Analyst",
            tools=[wikitool],
            goal="Analyze current industry trends, challenges and opportunities",
            backstory=dedent("""
                             As an Industry Analyst, your analysis will identify key trends,
                            challenges facing the industry, and potential opportunities that
                            could be leveraged during the meeting for strategic advantage.
                             """),
            verbose=True
        )
    
    def meeting_strategy_agent(self):
        return Agent(
            role="Meeting Strategy Advisor",
            tools=[wikitool],
            goal="Develop talking points, questions, and strategic angles for the meeting",
            backstory=dedent("""
                             As a Strategy Advisor, your expertise will guide the development of
                             talking points, insightful questions, and strategic angles
                             to ensure the meeting's objectives are achieved.
                             """
                ),
            verbose=True
        )
    
    def summary_and_breifing_agent(self):
        return Agent(
            role='Briefing Coordinator',
			goal='Compile all gathered information into a concise, informative briefing document',
			tools=[wikitool],
			backstory=dedent("""
					As the Briefing Coordinator, your role is to consolidate the research,
					analysis, and strategic insights."""),
			verbose=True
   )

### Next we define the tasks involed
We now need to define the tasks involved in our process and specify the agents to perform them

In [7]:
from crewai import Task

In [8]:
class MeetingPrepTasks():
    def research_task(self, agent, participants, context):
        return Task(
            description=dedent(f"""
				Conduct comprehensive research on each of the individuals and companies
				involved in the upcoming meeting. Gather information on recent
				news, achievements, professional background, and any relevant
				business activities.

				Participants: {participants}
				Meeting Context: {context}"""),
			expected_output=dedent("""
				A detailed report summarizing key findings about each participant
				and company, highlighting information that could be relevant for the meeting."""),
			async_execution=True,
			agent=agent
        )
    
    def industry_analysis_taks(self, agent, participants, context):
        return Task(
			description=dedent(f"""
				Analyze the current industry trends, challenges, and opportunities
				relevant to the meeting's context. Consider market reports, recent
				developments, and expert opinions to provide a comprehensive
				overview of the industry landscape.

				Participants: {participants}
				Meeting Context: {context}"""),
			expected_output=dedent("""
				An insightful analysis that identifies major trends, potential
				challenges, and strategic opportunities."""),
			async_execution=True,
			agent=agent
        )
    
    def meeting_strategy_task(self, agent, context, objective):
        return Task(
            description=dedent(f"""
                               Develop strategic talking points, questions, and discussion angles
                               for the meeting based on the research and industry analysis conducted

                               Meeting Context: {context}
                               Meeting Objective: {objective}
                               """),
            
            expected_output=dedent("""\
				Complete report with a list of key talking points, strategic questions
				to ask to help achieve the meetings objective during the meeting."""),
			agent=agent
            
        )
    
    def summary_and_briefing_task(self, agent, context, objective):
        return Task(
            description=dedent(f"""
                               Compile all the research findings, industry analysis, and strategic
				talking points into a concise, comprehensive briefing document for
				the meeting.
				Ensure the briefing is easy to digest and equips the meeting
				participants with all necessary information and strategies.

				Meeting Context: {context}
				Meeting Objective: {objective}
                               """),
            
            expected_output=dedent("""
				A well-structured briefing document that includes sections for
				participant bios, industry overview, talking points, and
				strategic recommendations."""),
			agent=agent

        )

In [9]:
from crewai import Crew

In [10]:
tasks = MeetingPrepTasks() # tasks for meeting preparation
agents = MeetingPrepAgents() # agents for meeting preparation

In [15]:
participants = "Geoffery Hinton"
context = "agxncy.ai is an ai startup that specializes in building special agents for innovative companies"
objective = "Detailed plan to make agxncy.ai a leading product and consultancy company"

In [16]:
# create agents
research_agent = agents.research_agent()
summary_and_briefing_agent = agents.summary_and_breifing_agent()
meeting_strategy_agent = agents.meeting_strategy_agent()
industry_analysis_agent = agents.industry_analysis_agent()

In [17]:
# create tasks
research = tasks.research_task(research_agent, participants, context)
industry_analysis = tasks.industry_analysis_taks(industry_analysis_agent, participants, context)
meeting_strategy = tasks.meeting_strategy_task(meeting_strategy_agent, context, objective)
summary_and_briefing = tasks.summary_and_briefing_task(summary_and_briefing_agent, context, objective)


In [18]:
crew = Crew(
	agents=[
		research_agent,
		industry_analysis_agent,
		meeting_strategy_agent,
		summary_and_briefing_agent
	],
	tasks=[
		research,
		industry_analysis,
		meeting_strategy,
		summary_and_briefing
	]
)

game = crew.kickoff()



> Entering new CrewAgentExecutor chain...
 Thought: Do I need to use a tool? Yes
Action: search
Action Input: Geoffery Hinton biography and recent newsNo good Wikipedia Search Result was found Do I need to use a tool? Yes
Action: search
Action Input: Geoffrey Hinton latest news and achievementsPage: Deep learning
Summary: Deep learning is the subset of machine learning methods based on artificial neural networks (ANNs) with representation learning. The adjective "deep" refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.Deep-learning architectures such as deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where 